In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
from tensorflow.image import resize
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, ZeroPadding2D
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomZoom, RandomRotation, SeparableConv2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Nadam, Adadelta

In [2]:
get_ipython().system('nvidia-smi')

Wed Apr 16 15:15:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   67C    P8              6W /  115W |       1MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Data load

In [ ]:
with h5py.File('./Data/GreenBit_Train_Original_Augmentation.hdf5', 'r') as hf:
    X_train = hf.get('x_train')[:]
    X_target = hf.get('x_target')[:]
    
with h5py.File('./Data/GreenBit_Test_Preprocessing.hdf5', 'r') as hf:
    X_test = hf.get('x_test')[:]
    X_label = hf.get('x_label')[:]

In [ ]:
print(X_train.shape, X_target.shape)
print(X_test.shape, X_label.shape)

## Data Check

In [ ]:
display(X_train[0])
display(X_test[0])

## image print

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(1, 2, 1)
plt.imshow(X_train[0], cmap='Greys')

plt.subplot(1, 2, 2)
plt.imshow(X_test[0], cmap='Greys')
plt.show()

print(X_target[0], X_label[0])

# Create Model

## Custom Model

In [ ]:
# # 하이퍼 파라미터 튜닝
# img_size_h = 224
# img_size_v = 224

# batch_size = 128
# epochs = 100
# learning_rate = 1e-5

# conv = 3
# pool = 2

# nb_class = 2

# _optimizer = Nadam(learning_rate=learning_rate)

# # 모델 쌓기
# model = Sequential()

# model.add(Input(shape=(img_size_h, img_size_v, 1)))

# model.add(Conv2D(32, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))
    
# model.add(Conv2D(32, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))
    
# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))
    
# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))

# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))

# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), padding='same'))
    
# model.add(Conv2D(128, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool)))
    
# model.add(Conv2D(128, (conv, conv), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool)))
# model.add(Dropout(0.3))
    
# model.add(Flatten())
# model.add(Dense(128, kernel_regularizer=regularizers.l2(learning_rate)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))

# model.add(Dense(nb_class))
# model.add(Activation('sigmoid'))

# # 모델의 실행 옵션을 설정합니다.
# model.compile(loss='binary_crossentropy', optimizer=_optimizer, metrics=['accuracy'])
# model.summary()

# model.save("./Model/GreenBit_Model.h5")

# hist = model.fit(X_train, X_target, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.25)

In [ ]:
score = model.evaluate(X_train, X_target, batch_size=batch_size, verbose=0)
print('Loss : {0:.4f}, Accuracy : {1:.4f} '.format(score[0], score[1]))

In [ ]:
# with tf.device('/device:GPU:2'):
score = model.evaluate(X_test, X_label, batch_size=batch_size, verbose=0)
print('Accuracy : {:.4f} '.format(score[1]))

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

## VGGNet

In [ ]:
# # 하이퍼 파라미터 튜닝
# img_size_h = 224
# img_size_v = 224

# batch_size = 128
# epochs = 100
# learning_rate = 1e-5

# conv = 3
# pool = 2

# nb_class = 2

# _optimizer = Nadam(learning_rate=learning_rate)

# # 모델 쌓기
# model = Sequential()

# model.add(Input(shape=(img_size_h, img_size_v, 1)))

# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(pool, pool), strides=2))
    
# model.add(Conv2D(128, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), strides=2))

# model.add(Conv2D(256, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), strides=2))

# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), strides=2))

# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (conv, conv), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(pool, pool), strides=2))
    
# model.add(Flatten())
# model.add(Dense(4096, kernel_regularizer=regularizers.l2(learning_rate)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(4096, kernel_regularizer=regularizers.l2(learning_rate)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(nb_class))
# model.add(Activation('sigmoid'))

# # 모델의 실행 옵션을 설정합니다.
# model.compile(loss='binary_crossentropy', optimizer=_optimizer, metrics=['accuracy'])
# model.summary()

# model.save("./Model/GreenBit_VGG_Model.h5")

# hist = model.fit(X_train, X_target, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.25)

## ResNet50

In [ ]:
# # 하이퍼 파라미터 튜닝
# img_size_h = 224
# img_size_v = 224

# batch_size = 128
# epochs = 100
# learning_rate = 1e-5

# conv = 3
# pool = 2

# nb_class = 2

# _optimizer = Nadam(learning_rate=learning_rate)

# # layer function
# def first_convolution(input_tensor):
#     x = ZeroPadding2D(padding=(3, 3))(input_tensor)
#     x = Conv2D(filters=64, strides=(2, 2), kernel_size=(7,7))(x)
#     x = Activation('relu')(x)
    
#     return x

# def first_pooling(input_tensor):
#     x = ZeroPadding2D(padding=(1, 1))(input_tensor)
#     x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
#     return x

# def residual_block(input_tensor, kernel_size, num_filters):
#     x = Conv2D(kernel_size=(1, 1), filters=num_filters//4)(input_tensor)
#     x = Activation('relu')(x)
    
#     x = Conv2D(kernel_size=kernel_size, filters=num_filters//4, padding='same')(x)
#     x = Activation('relu')(x)
    
#     x = Conv2D(kernel_size=(1, 1), filters=num_filters)(x)
#     x = Add()([input_tensor, x])
    
#     x = Activation('relu')(x)
    
#     return x

# def downsize_block(input_tensor, kernel_size, strides, num_filters):
#     x = Conv2D(kernel_size=(1, 1), filters=num_filters//4, strides=strides)(input_tensor)
#     x = Activation('relu')(x)
    
#     x = Conv2D(kernel_size=kernel_size, filters=num_filters//4, padding='same')(x)
#     x = Activation('relu')(x)
    
#     x = Conv2D(kernel_size=(1, 1), filters=num_filters)(x)
    
#     shortcut = Conv2D(kernel_size=(1, 1), filters=num_filters, strides=strides)(input_tensor)
    
#     x = Add()([shortcut, x])
#     output_tensor = Activation('relu')(x)
    
#     return output_tensor

# # 모델 쌓기
# input_tensor = Input(shape=(224, 224, 1))

# x = first_convolution(input_tensor)

# x = first_pooling(x)
# x = downsize_block(x, (3, 3), (1, 1), 256)
# x = residual_block(x, (3, 3), 256)
# x = residual_block(x, (3, 3), 256)

# x = downsize_block(x, (3, 3), (2, 2), 512)
# x = residual_block(x, (3, 3), 512)
# x = residual_block(x, (3, 3), 512)
# x = residual_block(x, (3, 3), 512)

# x = downsize_block(x, (3, 3), (2, 2), 1024)
# x = residual_block(x, (3, 3), 1024)
# x = residual_block(x, (3, 3), 1024)
# x = residual_block(x, (3, 3), 1024)
# x = residual_block(x, (3, 3), 1024)
# x = residual_block(x, (3, 3), 1024)

# x = downsize_block(x, (3, 3), (2, 2), 2048)
# x = residual_block(x, (3, 3), 2048)
# x = residual_block(x, (3, 3), 2048)

# x = GlobalMaxPooling2D()(x)
# x = Dense(nb_class, activation='sigmoid')(x)

# model = Model(inputs=input_tensor, outputs=x)

# # 모델의 실행 옵션을 설정합니다.
# model.compile(loss='binary_crossentropy', optimizer=_optimizer, metrics=['accuracy'])
# model.summary()

# model.save("./Model/GreenBit_ResNet_Model.h5")

# hist = model.fit(X_train, X_target, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.25)

## SlimResCNN

In [ ]:
# 하이퍼 파라미터 튜닝
img_size_h = 224
img_size_v = 224

batch_size = 128
epochs = 30
learning_rate = 1e-5

conv = 3
pool = 2

nb_class = 2

_optimizer = Nadam(learning_rate=learning_rate)

# layer function
def first_convolution(input_tensor):
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def residual_block_type_B(input_tensor, out_channal):
    x = Conv2D(filters=out_channal, kernel_size=3, strides=1, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=out_channal, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    return x

def downsize_block_type_B(input_tensor, out_channal):
    x = Conv2D(filters=out_channal, kernel_size=3, strides=2)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=out_channal, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    return x

def Slim_Res_CNN():
    input_tensor = Input(shape=(None,None,1))
    
    conv1 = first_convolution(input_tensor)
    
    conv2 = residual_block_type_B(conv1, 32)
    conv2 = residual_block_type_B(conv2, 32)
    conv2 = residual_block_type_B(conv2, 32)
    
    conv3_1 = downsize_block_type_B(conv2, 64)
    conv3_2 = residual_block_type_B(conv3_1, 64)
    conv3_2 = residual_block_type_B(conv3_2, 64)
    
    conv4_1 = downsize_block_type_B(conv3_2, 128)
    conv4_2 = residual_block_type_B(conv4_1, 128)
    conv4_2 = residual_block_type_B(conv4_2, 128)
    
    avg_pool = GlobalAveragePooling2D()(conv4_2)
    out = Dense(nb_class, activation='sigmoid')(avg_pool)
    
    model = Model(inputs=input_tensor, outputs=out)
    
    return model

In [ ]:
# # layer function
# def conv3x3(x, out_channel, strides):
#     x = tf.keras.layers.Conv2D(out_channel, kernel_size=3,strides=strides,padding='same')(x)

#     return x

# def conv3x3_with_dropout(x, out_channel, strides):
#     x = conv3x3(x, out_channel, strides)
#     x = BatchNormalization()(x)
#     x = tf.keras.layers.Activation('relu')(x)
     
#     return x

# def pad_depth(x, desired_channels):
#     new_channels = desired_channels - x.shape.as_list()[-1]
#     output = tf.identity(x)
#     repetitions = new_channels/x.shape.as_list()[-1]
#     for _ in range(int(repetitions)):
#         zeroTensors = tf.zeros_like(x, name='pad_depth1')
#         output = tf.keras.backend.concatenate([output, zeroTensors])
#     return output

# def Resblock(inputs, out_channel,downsample=False):
    
#     if downsample:
#         x = conv3x3_with_dropout(inputs, out_channel,strides=2)
#         x = conv3x3(x, out_channel,strides=1)
        
#         down_sampled = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(inputs)
#         equal = pad_depth(down_sampled,out_channel)
        
#         out = tf.keras.layers.Add()([x,equal])
        
#     else:
#         x = conv3x3_with_dropout(inputs, out_channel,strides=1)
#         x = conv3x3(x, out_channel, strides=1)
        
#         out = tf.keras.layers.Add()([x, inputs])
        
#     return out

# def Conv1(x):
#     x = conv3x3(x,32,strides=1)
#     x = tf.keras.layers.Activation('relu')(x)
#     return x

# def Conv2(x):
#     for i in range(3):
#         x = Resblock(x, 32)
#     return x

# def Conv3(x):
#     x = Resblock(x, 64, True)
#     for i in range(2):
#         x = Resblock(x, 64)
#     return x

# def Conv4(x):
#     x = Resblock(x, 128, True)
#     for i in range(2):
#         x = Resblock(x, 128)
#     return x

# def GAP(x):
#     gap = tf.reduce_mean(x, axis=[1,2],keepdims=True)
#     return gap

# def SlimResNet():
#     inputs = Input(shape=(None,None,1))
    
#     conv1 = Conv1(inputs)
#     conv2 = Conv2(conv1)
#     conv3 = Conv3(conv2)
#     conv4 = Conv4(conv3)
    
#     x = tf.keras.layers.GlobalAveragePooling2D()(conv4)
#     out = tf.keras.layers.Dense(2,activation='softmax')(x)
    
#     model = tf.keras.Model(inputs=inputs, outputs=out)
    
#     return model


In [ ]:
model = Slim_Res_CNN()

model.compile(loss='binary_crossentropy', optimizer=_optimizer, metrics=['accuracy'])
model.summary()

model.save("./Model/GreenBit_SlimResNet_Model.h5")

hist = model.fit(X_train, X_target, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.25)


In [ ]:
test = model.predict(X_test)

In [ ]:
cnt = 0
for i in range(len(test)):
    print(f"{i+1}번째 Test Data의 실제 값 : {X_label[i][1]}")
    print(f"{i+1}번째 Test Data의 예측 값 : {test[i][1]:.4f}")
    if X_label[i][1] == 1:
        if test[i][1] >= 0.5:
            cnt += 1
    else:
        if test[i][1] < 0.5:
            cnt += 1

print(f"{cnt / len(test):.4f}")